In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import os

In [3]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [4]:
DEVICE

device(type='cuda')

In [5]:
BATCH_SIZE = 256
EPOCH = 30

In [6]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

In [7]:
'''
Compose()는 이미지 데이터 전처리 증강등의 과정에서 사용됨

'''
data_transform ={
    'train': transforms.Compose([
        transforms.Resize([64,64]),
        transforms.RandomHorizontalFlip(p=0.5), # p=반전되는 비율
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229,0.224,0.225])
    ]),
    'val' : transforms.Compose([
        transforms.Resize([64,64]),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229,0.224,0.225])
    ])
}

data_dir = '../0_data'

image_datasets = {x : ImageFolder(root=os.path.join(data_dir, x), 
            transform=data_transform[x]) for x in ['train', 'val']}

dataloaders = {x : torch.utils.data.DataLoader(image_datasets[x], 
            batch_size=BATCH_SIZE, shuffle=True, num_workers=4) for x in ['train', 'val']}

dataset_sizes = {x : len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

### Pre-trained Model 불러오기

In [8]:
from torchvision import models # 왜? 여기에서

In [9]:
# resnet = models.resnet50(pretrained=True)
effb1= models.efficientnet_b1(pretrained=True)
num_ftrs = effb1.classifier[1].in_features # 이렇게까지 찾아야하나...
effb1.classifier[1] = nn.Linear(num_ftrs, len(class_names))

effb1 = effb1.to(DEVICE)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(filter(lambda p : p.requires_grad, 
                        effb1.parameters()), lr=0.001)

from torch.optim import lr_scheduler

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)


### 만약 freeze를 해야겠다면

In [11]:
cnt = 0
for child in effb1.children():
    cnt += 1
    if cnt < 0:
        for param in child.parameters():
            param.requires_grad = False

In [12]:
import copy
import time

In [13]:
def train_eff(model, criterion, optimizer, scheduler, num_epochs):

    best_model = copy.deepcopy(model.state_dict()) # 베스트모델
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'--------epoch {epoch+1}--------')
        since = time.time()

        for phase in ['train', 'val']: # 페이즈 순서
            if phase == 'train':
                model.train() # 페이즈에 맞게 모드 설정 | 기억은 안 나지만 가중치 변경이 가능하도록
            elif phase == 'val' :
                model.eval() # 페이즈에 맞게 모드 설정 | 기억은 안 나지만 가중치 고정

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]: # 두개를 동시에 뱉는다!
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'): # 모델의 Gradient를 업데이트
                    outputs = model(inputs) # 아웃풋... pred는 더 뒤에
                    _, preds = torch.max(outputs, 1) # 가장 높은 것을 가져온다.
                    loss = criterion(outputs, labels) # pred와 gt의 사이 값을 구한다.

                    if phase == 'train': # 파라미터 업데이트 구간이라는데...
                        loss.backward()
                        optimizer.step() 
            
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                scheduler.step()
                l_r = [x['lr'] for x in optimizer_ft.param_groups] # 7에프크 마다 lr을 수정한다.
                print('learning rate :', l_r)

            epoch_loss = running_loss/dataset_sizes[phase]
            epoch_acc = running_corrects.double()/dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'
                                .format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc :
                best_acc = epoch_acc
                best_model = copy.deepcopy(model.state_dict())

        
        time_took = time.time() - since
        print('Completed in {:.0f}m, {:.0f}s'
                            .format(time_took//60, time_took%60))
        
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model)

    return model



In [14]:
model_effb1 = train_eff(effb1, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=EPOCH)

--------epoch 1--------
learning rate : [0.001]
train Loss: 0.7270 Acc: 0.8034
val Loss: 0.1609 Acc: 0.9479
Completed in 0m, 45s
--------epoch 2--------
learning rate : [0.001]
train Loss: 0.1375 Acc: 0.9558
val Loss: 0.1036 Acc: 0.9665
Completed in 0m, 33s
--------epoch 3--------
learning rate : [0.001]
train Loss: 0.0896 Acc: 0.9710
val Loss: 0.0649 Acc: 0.9783
Completed in 0m, 34s
--------epoch 4--------
learning rate : [0.001]
train Loss: 0.0648 Acc: 0.9778
val Loss: 0.0839 Acc: 0.9729
Completed in 0m, 34s
--------epoch 5--------
learning rate : [0.001]
train Loss: 0.0605 Acc: 0.9798
val Loss: 0.0894 Acc: 0.9697
Completed in 0m, 34s
--------epoch 6--------
learning rate : [0.001]
train Loss: 0.0527 Acc: 0.9829
val Loss: 0.0624 Acc: 0.9805
Completed in 0m, 34s
--------epoch 7--------
learning rate : [0.0001]
train Loss: 0.0440 Acc: 0.9856
val Loss: 0.0397 Acc: 0.9871
Completed in 0m, 34s
--------epoch 8--------
learning rate : [0.0001]
train Loss: 0.0203 Acc: 0.9929
val Loss: 0.0290

In [15]:
torch.save(model_effb1, 'effb1.pt')

### 평가 하기

In [16]:
def evaluate(model, test_loader):
    model.eval()  
    test_loss = 0 
    correct = 0   
    
    with torch.no_grad(): 
        for data, target in test_loader:  
            data, target = data.to(DEVICE), target.to(DEVICE)  
            output = model(data) 
            
            test_loss += F.cross_entropy(output,target, reduction='sum').item() 
 
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item() 
   
    test_loss /= len(test_loader.dataset) 
    test_accuracy = 100. * correct / len(test_loader.dataset) 
    return test_loss, test_accuracy  

In [17]:
transform_effb1 = transforms.Compose([
        transforms.Resize([64,64]),  
        transforms.RandomCrop(52),  
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
    ])

In [18]:
test_effb1 = ImageFolder(root='../0_data/test', transform=transform_effb1) 
test_loader_effb1 = torch.utils.data.DataLoader(test_effb1, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [19]:
resnet50=torch.load('effb1.pt') 
resnet50.eval()  
test_loss, test_accuracy = evaluate(resnet50, test_loader_effb1)

print('effb1 test acc:  ', test_accuracy)

effb1 test acc:   99.32292136860161
